# A2A Client example

In order to run this example, you need to run the server:

```
python a2a_min/examples/base/server_example.py
```

or using the utility script:
```
a2a_min/scripts/run_server_example_base.sh
```

### Initialize

In [1]:
import sys
from pathlib import Path
from uuid import uuid4

In [2]:
# Add the project's root directory to the Python path
import sys

PROJECT_ROOT = Path.cwd().resolve().parent.parent.parent
print(f"PROJECT_ROOT = {PROJECT_ROOT}")
if PROJECT_ROOT not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print(f"sys.path: {sys.path}")

PROJECT_ROOT = /Users/kqrw311/workspace/aixplore/a2a_min
sys.path: ['/Users/kqrw311/.pyenv/versions/3.12.2/lib/python312.zip', '/Users/kqrw311/.pyenv/versions/3.12.2/lib/python3.12', '/Users/kqrw311/.pyenv/versions/3.12.2/lib/python3.12/lib-dynload', '', '/Users/kqrw311/workspace/aixplore/a2a_min/.venv/lib/python3.12/site-packages', '__editable__.a2a_min-0.1.0.finder.__path_hook__', '/Users/kqrw311/workspace/aixplore/a2a_min']


In [3]:
from a2a_min.base.client.card_resolver import A2ACardResolver
from a2a_min.base.client.client import A2AClient
from a2a_min.base.types import Message, TaskSendParams, TaskState, TextPart

In [4]:
A2A_SERVER_URL = "http://localhost:10000"

### Get A2A server's card

In [5]:
card_resolver = A2ACardResolver(A2A_SERVER_URL)
card = card_resolver.get_agent_card()

print(card.model_dump_json(exclude_none=True, indent=2))

{
  "name": "Dummy Agent",
  "description": "Pretneds to be a usefull agent...",
  "url": "http://localhost:10000/",
  "version": "1.0.0",
  "capabilities": {
    "streaming": true,
    "pushNotifications": true,
    "stateTransitionHistory": false
  },
  "defaultInputModes": [
    "text"
  ],
  "defaultOutputModes": [
    "text"
  ],
  "skills": [
    {
      "id": "dummy_agent",
      "name": "Dummy Agent",
      "description": "Pretneds to be a usefull agent...",
      "tags": [
        "dummy",
        "fake"
      ],
      "examples": [
        "Are you a real agent?",
        "What is the meaning of life?"
      ]
    }
  ]
}


### Create an A2A client

In [6]:
client = A2AClient(agent_card=card)

In [7]:
def to_json(data):
    if hasattr(data, "model_dump_json"):
        return data.model_dump_json(indent=2, exclude_none=True)
    return data.json(indent=2, exclude_none=True)

In [8]:



async def completeTask(a2a_client: A2AClient, prompt: str, taskId, sessionId):
    """ Run a task with the given prompt """
    if prompt == ":q" or prompt == "quit":
        return False

    message = Message(role="user", parts=[TextPart(text=prompt)])
    tasK_send_params = TaskSendParams(id=taskId, sessionId=sessionId, acceptedOutputModes=["text"], message=message)

    taskResult = await a2a_client.send_task(tasK_send_params)
    print(f"\n{to_json(taskResult)}")

    ## if the result is that more input is required, loop again.
    state = TaskState(taskResult.result.status.state)
    if state.name == TaskState.INPUT_REQUIRED.name:
        return await completeTask(a2a_client, prompt, taskId, sessionId)
    else:
        return False    # We are done with this task.


In [9]:
prompt = "What do you want to send to the agent? (:q or quit to exit)"

continue_loop = True
sessionId = uuid4().hex
while continue_loop:
    taskId = uuid4().hex
    print("=========  starting a new task ======== ")
    continue_loop = await completeTask(client, prompt, taskId, sessionId)

=========  starting a new task ======== 

{
  "jsonrpc": "2.0",
  "id": "091cb1627d4244df8c6e9f235ab8ddc7",
  "result": {
    "id": "674ec0509bd24dc0af420a864bded182",
    "sessionId": "213b6e35d8fc486f9ece5015ff28e49b",
    "status": {
      "state": "completed",
      "timestamp": "2025-04-15T13:29:39.375689"
    },
    "artifacts": [
      {
        "parts": [
          {
            "type": "text",
            "text": "We are unable to process your request at the moment. Please try again."
          }
        ],
        "index": 0
      }
    ],
    "history": []
  }
}
